In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
done

# All requested packages already installed.

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
done

# All requested packages already installed.

Libraries imported.


In [2]:
#swiss_data = pd.read_json("https://geo.nyu.edu/download/file/stanford-cm591kd0445-geojson.json")
#swiss_data = pd.read_json("https://raw.githubusercontent.com/zauberware/postal-codes-json-xml-csv/master/data/CH/zipcodes.ch.json")

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
zurich_data = pd.read_json(body, orient='values')
zurich_data.head()

,type,name,features
0,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
1,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
2,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
3,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
4,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."


In [5]:
zurich_data_features = zurich_data['features']
zurich_data_features.head()

0    {'type': 'Feature', 'geometry': {'type': 'Poin...
1    {'type': 'Feature', 'geometry': {'type': 'Poin...
2    {'type': 'Feature', 'geometry': {'type': 'Poin...
3    {'type': 'Feature', 'geometry': {'type': 'Poin...
4    {'type': 'Feature', 'geometry': {'type': 'Poin...
Name: features, dtype: object

In [6]:
#swiss_data.sort_values('place',ascending=False)
#swiss_data

In [7]:
#zurich_data = swiss_data[swiss_data['place'] == 'Zürich'].reset_index(drop=True)
#zurich_data

In [8]:
# define the dataframe columns
column_names = ['Borough','Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [9]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [10]:
for data in zurich_data_features:
    neighborhood_name = data['properties']['name'] 

    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': "Zürich",
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, 
                                           ignore_index=True)
    

In [11]:
neighborhoods.shape

(22, 4)

In [12]:
print('The dataframe has {} neighborhoods.'.format(
        len(neighborhoods['Neighborhood'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 22 neighborhoods.


In [13]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude
0,Zürich,Hirslanden,47.363057,8.570403
1,Zürich,Witikon,47.359977,8.600887
2,Zürich,Leimbach,47.331480,8.509313
3,Zürich,Wollishofen,47.341491,8.532956
4,Zürich,Enge,47.360931,8.533140
5,Zürich,Riesbach,47.354570,8.556241
6,Zürich,Affoltern,47.423281,8.509221
7,Zürich,Seebach,47.423695,8.542830
8,Zürich,Oerlikon,47.409348,8.545117
9,Zürich,Höngg,47.407904,8.496424


In [14]:
address = 'Zurich, Zurich'

geolocator = Nominatim(user_agent="zh_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Zurich are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Zurich are 47.3744489, 8.5410422.


In [15]:
# create map of Zurich using latitude and longitude values
map_zh = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood, borough in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood'], neighborhoods['Borough']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zh)  
    
map_zh

## Foursquare Credentials and Version

In [43]:
CLIENT_ID = 'D0WVJEV1OWOCGMZ23LW3AV2PUIHF2QXIWCLWQS0V0NBMWV0Q' # your Foursquare ID
CLIENT_SECRET = 'F54CW4QHMTFO23ZZ0LTCEC4XU42PVEEADHNJCNPNDTKVVXIQ' # your Foursquare Secret
ACCESS_TOKEN = 'EUXOMD4V042AJ0VBANQVMRQDPSALDTJLWFD4CVMNIMNYIPVE' # Foursquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: D0WVJEV1OWOCGMZ23LW3AV2PUIHF2QXIWCLWQS0V0NBMWV0Q
CLIENT_SECRET:F54CW4QHMTFO23ZZ0LTCEC4XU42PVEEADHNJCNPNDTKVVXIQ


## Exploring Zurich neighborhoods

In [17]:
neighborhoods.loc[0, 'Neighborhood']

'Hirslanden'

In [18]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hirslanden are 47.3630568387, 8.5704034113.


# Get the top 100 venues that are in Hirslanden within a radius of 500 meters.

In [19]:
# Create the GET request URL
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=D0WVJEV1OWOCGMZ23LW3AV2PUIHF2QXIWCLWQS0V0NBMWV0Q&client_secret=F54CW4QHMTFO23ZZ0LTCEC4XU42PVEEADHNJCNPNDTKVVXIQ&v=20180605&ll=47.3630568387,8.5704034113&radius=500&limit=100'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '606e1958c190c03a959b5726'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Kreis 7',
  'headerFullLocation': 'Kreis 7, Zürich',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 47.3675568432, 'lng': 8.577034546600979},
   'sw': {'lat': 47.358556834199995, 'lng': 8.56377227599902}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c6b93162d9976b016990c3a',
       'name': 'Marcellino, Pane e Vino',
       'location': {'address': 'Forchstrasse 168',
        'lat': 47.360202,
        'lng': 8.565569,
        'labeledLatLngs': [{'label': 'display',
          'lat': 47.360202,
          'lng': 8.

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,"Marcellino, Pane e Vino",Mediterranean Restaurant,47.360202,8.565569
1,Wilder Mann,Swiss Restaurant,47.360341,8.566796
2,Ristorante Antiquario,Italian Restaurant,47.360676,8.566550
3,Klusplatz,Plaza,47.364025,8.566751
4,Bistro Le Puy,French Restaurant,47.359382,8.568897


In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


## Explore neighborhoods in Zurich

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
zurich_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                latitudes=neighborhoods['Latitude'],
                                longitudes=neighborhoods['Longitude']
                                )

Hirslanden
Witikon
Leimbach
Wollishofen
Enge
Riesbach
Affoltern
Seebach
Oerlikon
Höngg
Unterstrass
Wipkingen
Schwamendingen
Oberstrass
Altstetten
Aussersihl
Altstadt
Fluntern
Hottingen
Albisrieden
Wiedikon
Industriequartier


In [26]:
print(zurich_venues.shape)
zurich_venues.head()

(518, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hirslanden,47.363057,8.570403,"Marcellino, Pane e Vino",47.360202,8.565569,Mediterranean Restaurant
1,Hirslanden,47.363057,8.570403,Wilder Mann,47.360341,8.566796,Swiss Restaurant
2,Hirslanden,47.363057,8.570403,Ristorante Antiquario,47.360676,8.566550,Italian Restaurant
3,Hirslanden,47.363057,8.570403,Klusplatz,47.364025,8.566751,Plaza
4,Hirslanden,47.363057,8.570403,Bistro Le Puy,47.359382,8.568897,French Restaurant


In [27]:
# Venues per neighborhood
zurich_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Affoltern,4,4,4,4,4,4
Albisrieden,5,5,5,5,5,5
Altstadt,89,89,89,89,89,89
Altstetten,21,21,21,21,21,21
Aussersihl,63,63,63,63,63,63
Enge,31,31,31,31,31,31
Fluntern,8,8,8,8,8,8
Hirslanden,9,9,9,9,9,9
Hottingen,2,2,2,2,2,2


In [28]:
# Check unique categories we received for Zurich venues
print('There are {} unique categories.'.format(len(zurich_venues['Venue Category'].unique())))

There are 145 unique categories.


## 3. Analyze Each Neighborhood

In [29]:
zurich_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hirslanden,47.363057,8.570403,"Marcellino, Pane e Vino",47.360202,8.565569,Mediterranean Restaurant
1,Hirslanden,47.363057,8.570403,Wilder Mann,47.360341,8.566796,Swiss Restaurant
2,Hirslanden,47.363057,8.570403,Ristorante Antiquario,47.360676,8.566550,Italian Restaurant
3,Hirslanden,47.363057,8.570403,Klusplatz,47.364025,8.566751,Plaza
4,Hirslanden,47.363057,8.570403,Bistro Le Puy,47.359382,8.568897,French Restaurant
5,Hirslanden,47.363057,8.570403,VBZ Klusplatz,47.364090,8.566398,Tram Station
6,Hirslanden,47.363057,8.570403,VBZ Wetlistrasse,47.360079,8.568567,Tram Station
7,Hirslanden,47.363057,8.570403,Sonnenberg,47.366290,8.567369,Park
8,Hirslanden,47.363057,8.570403,California House,47.365324,8.564940,Hotel
9,Witikon,47.359977,8.600887,Coop,47.360204,8.594794,Supermarket


In [30]:
zurich_venues['Venue Category'].unique()

array(['Mediterranean Restaurant', 'Swiss Restaurant',
       'Italian Restaurant', 'Plaza', 'French Restaurant', 'Tram Station',
       'Park', 'Hotel', 'Supermarket', 'Bus Station', 'Soccer Field',
       'Grocery Store', 'Tennis Court', "Dentist's Office",
       'Moroccan Restaurant', 'Restaurant', 'Music Venue', 'Cheese Shop',
       'Community Center', 'Irish Pub', 'Pool', 'Café',
       'Salon / Barbershop', 'Gas Station', 'History Museum',
       'Sauna / Steam Room', 'BBQ Joint', 'Thai Restaurant',
       'Burger Joint', 'Cupcake Shop', 'Bar', 'Train Station',
       'Middle Eastern Restaurant', 'Chinese Restaurant', 'Coffee Shop',
       'Japanese Restaurant', 'Beach', 'Movie Theater', 'Garden',
       'Bakery', 'Taverna', 'Seafood Restaurant', 'Art Museum', 'Museum',
       'Mexican Restaurant', 'Beer Garden', 'Wine Bar', 'Playground',
       'Sushi Restaurant', 'Food Court', 'Department Store',
       'Athletics & Sports', 'Pizza Place', 'Indian Restaurant',
       'Bowling

In [64]:
neigh_with_train =zurich_venues[zurich_venues['Venue Category'] == 'Train Station' ].reset_index(drop=True)
neigh_with_train

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Enge,47.360931,8.533140,Bahnhof Zürich Enge,47.364305,8.530746,Train Station
1,Affoltern,47.423281,8.509221,Bahnhof Zürich Affoltern,47.420950,8.507619,Train Station
2,Oerlikon,47.409348,8.545117,Bahnhof Oerlikon,47.411978,8.544489,Train Station
3,Aussersihl,47.379690,8.521656,Zürich Vorbahnhof,47.382500,8.523395,Train Station


In [83]:
# Search for supermarkets nearby
search_query = 'Store'
latitude = neigh_with_train.loc[3,'Venue Latitude']
longitude = neigh_with_train.loc[3,'Venue Longitude']
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
print(url)

https://api.foursquare.com/v2/venues/search?client_id=D0WVJEV1OWOCGMZ23LW3AV2PUIHF2QXIWCLWQS0V0NBMWV0Q&client_secret=F54CW4QHMTFO23ZZ0LTCEC4XU42PVEEADHNJCNPNDTKVVXIQ&ll=47.382500437679305,8.52339506149292&oauth_token=EUXOMD4V042AJ0VBANQVMRQDPSALDTJLWFD4CVMNIMNYIPVE&v=20180605&query=Store&radius=500&limit=100


In [84]:
res = requests.get(url).json()
res

{'meta': {'code': 200, 'requestId': '606e2938d9499c362a0a198e'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4b05888bf964a520f7cc22e3',
    'name': 'Freitag Flagship Store',
    'location': {'address': 'Geroldstr. 17',
     'lat': 47.3857878244881,
     'lng': 8.519485804873634,
     'labeledLatLngs': [{'label': 'display',
       'lat': 47.3857878244881,
       'lng': 8.519485804873634}],
     'distance': 469,
     'postalCode': '8005',
     'cc': 'CH',
     'city': 'Zürich',
     'state': 'Zürich',
     'country': 'Schweiz',
     'formattedAddress': ['Geroldstr. 17', '8005 Zürich']},
    'categories': [{'id': '4bf58dd8d48988d102951735',
      'name': 'Accessories Store',
      'pluralName': 'Accessories Stores',
      'shortName': 'Accessories',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_accessories_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1617832248

In [85]:
# assign relevant part of JSON to venues
stores = res['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(stores)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4b05888bf964a520f7cc22e3,Freitag Flagship Store,"[{'id': '4bf58dd8d48988d102951735', 'name': 'A...",v-1617832248,False,Geroldstr. 17,47.385788,8.519486,"[{'label': 'display', 'lat': 47.3857878244881,...",469,8005,CH,Zürich,Zürich,Schweiz,"[Geroldstr. 17, 8005 Zürich]",NaN
1,4bee827b5e7b9c74760cb58d,Hum Records Store,"[{'id': '4bf58dd8d48988d1fe941735', 'name': 'M...",v-1617832248,False,Ankerstrasse 11,47.378993,8.525167,"[{'label': 'display', 'lat': 47.378993, 'lng':...",412,8804,CH,Zürich,Zürich,Schweiz,"[Ankerstrasse 11, 8804 Zürich]",NaN
2,4eb5156fe5fa17fc869ac4d9,Kahoona & Somea Check Out Store,"[{'id': '4bf58dd8d48988d104951735', 'name': 'B...",v-1617832248,False,NaN,47.380012,8.527696,"[{'label': 'display', 'lat': 47.38001155889453...",426,NaN,CH,NaN,NaN,Schweiz,NaN,NaN
3,57b30b55cd10867326048c12,deus Pop Up store,"[{'id': '4bf58dd8d48988d106951735', 'name': 'M...",v-1617832248,False,NaN,47.386740,8.522605,"[{'label': 'display', 'lat': 47.38674, 'lng': ...",475,8005,CH,Zürich,Zürich,Schweiz,[8005 Zürich],NaN
4,55c0e849498ef4ba148ee433,EGO Movement E-Bike Store,"[{'id': '4bf58dd8d48988d115951735', 'name': 'B...",v-1617832248,False,"Geroldstrasse 23, 8005 Zürich",47.384995,8.518019,"[{'label': 'display', 'lat': 47.38499461234668...",491,8005,CH,Zürich,Zürich,Schweiz,"[Geroldstrasse 23, 8005 Zürich (Frau Gerolds G...",Frau Gerolds Garten


In [37]:
# one hot encoding
zurich_onehot = pd.get_dummies(zurich_venues[['Venue Category']], prefix="", prefix_sep="")
#zurich_onehot = pd.get_dummies(neigh_with_train[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
zurich_onehot['Neighborhood'] = zurich_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [zurich_onehot.columns[-1]] + list(zurich_onehot.columns[:-1])
zurich_onehot = zurich_onehot[fixed_columns]

zurich_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Argentinian Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Burger Joint,Burrito Place,Bus Station,Business Service,Café,Cambodian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Convenience Store,Cupcake Shop,Dance Studio,Dentist's Office,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Doner Restaurant,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Fondue Restaurant,Food,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Kebab Restaurant,Kids Store,Korean Restaurant,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paella Restaurant,Park,Pedestrian Plaza,Peruvian Restaurant,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Taco Place,Taverna,Tennis Court,Thai Restaurant,Theater,Tibetan Restaurant,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Water Park,Wine Bar
0,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hirslanden,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
zurich_onehot.shape

(518, 146)

In [39]:
# Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
zurich_grouped = zurich_onehot.groupby('Neighborhood').mean().reset_index()
zurich_grouped

,Neighborhood,Accessories Store,American Restaurant,Argentinian Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Burger Joint,Burrito Place,Bus Station,Business Service,Café,Cambodian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Concert Hall,Convenience Store,Cupcake Shop,Dance Studio,Dentist's Office,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Doner Restaurant,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Fondue Restaurant,Food,Food Court,Food Truck,French Restaurant,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Kebab Restaurant,Kids Store,Korean Restaurant,Light Rail Station,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paella Restaurant,Park,Pedestrian Plaza,Peruvian Restaurant,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Taco Place,Taverna,Tennis Court,Thai Restaurant,Theater,Tibetan Restaurant,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Water Park,Wine Bar
0,Affoltern,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Albisrieden,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,

In [40]:
zurich_grouped.shape

(22, 146)

In [41]:
num_top_venues = 5

for hood in zurich_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = zurich_grouped[zurich_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Affoltern----
                venue  freq
0         Supermarket  0.25
1  Athletics & Sports  0.25
2       Train Station  0.25
3    Department Store  0.25
4                 Pub  0.00


----Albisrieden----
           venue  freq
0    Bus Station   0.2
1  Grocery Store   0.2
2   Tram Station   0.2
3     Playground   0.2
4         Bakery   0.2


----Altstadt----
              venue  freq
0              Café  0.08
1               Bar  0.08
2  Swiss Restaurant  0.07
3             Plaza  0.07
4             Hotel  0.06


----Altstetten----
         venue  freq
0        Hotel  0.10
1  Bus Station  0.05
2         Pool  0.05
3          Gym  0.05
4  Supermarket  0.05


----Aussersihl----
                venue  freq
0                 Bar  0.16
1    Swiss Restaurant  0.08
2          Restaurant  0.06
3                Café  0.05
4  Italian Restaurant  0.05


----Enge----
                venue  freq
0                Park  0.16
1          Restaurant  0.10
2  Italian Restaurant  0.06
3         Superm